In [2]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP
import os

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
sys.path.append('/home/UCSD/cobra_utils')
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)

/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/sporeme/bacillusme/__init__.py


### Gene inactivation

In [3]:
inactivation = pandas.read_csv("./cytoscape/base_inactivation.csv")
inactivation["blockage"] = False
inactivation

,SOURCE,TARGET,blockage
0,nrdE,nrdF,False
1,nrdE,trxA,False
2,nrdE,trxB,False
3,nrdF,nrdE,False
4,nrdF,trxA,False
...,...,...,...
95,sufU,ilvD,False
96,sufU,mccB,False
97,sufU,mccA,False
98,sufU,ilvC,False


In [4]:
map = {(v["SOURCE"],v["TARGET"]):idx for idx,v in inactivation.iterrows()}

### Gene blockage

In [7]:
blockage = pandas.read_csv("./cytoscape/base_blockage.csv")
blockage

,SOURCE,TARGET
0,alsS,ilvD
1,alsS,ilvC
2,pdhA,pdhB
3,pdhA,pdhC
4,pdhB,pdhC
5,pdhB,pdhA
6,ilvC,ilvD
7,ilvD,ilvC
8,pdhD,pdhB
9,pdhD,pdhC


In [8]:
for _,v in blockage.iterrows():
    edge = (v["SOURCE"],v["TARGET"])
    idx = map.get(edge,None)
    if idx is None:
        print(edge)
        continue
    inactivation.loc[idx,"blockage"] = True

### To table

In [9]:
known_depletions = [g[0].lower() + g[1:] for g in list(pd.read_csv('proteomics/known_depletions.txt')['protein'].values)]
confirmed_depletions = ['pckA','purL','ilvB','ilvE','citZ','acsA','etfA','acoC','sigF','yxbC','lutC',
                  'argJ','hpf']

In [10]:
def get_properties(gene):
    return {
        "confirmed" : True if gene in confirmed_depletions else False,
        "known" : True if gene in known_depletions else False
    }

In [11]:
dct = {}
counter = 0
for idx,row in inactivation.iterrows():
    i,j = row[["SOURCE","TARGET"]]
    if i == j:
        continue
    source_properties = {"SOURCE_"+k:v for k,v in get_properties(i).items()}
    target_properties = {"TARGET_"+k:v for k,v in get_properties(j).items()}


    tmp = {k:v for k,v in row.items()}
    tmp.update(source_properties)
    tmp.update(target_properties)
    dct[counter] = tmp
    counter += 1

In [12]:
properties = pandas.DataFrame.from_dict(dct).T

In [13]:
properties.set_index("SOURCE").to_csv("./cytoscape/base_network.csv")